In [2]:
!pip install -q kaggle

In [3]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [4]:
! kaggle datasets download paramaggarwal/fashion-product-images-small

Dataset URL: https://www.kaggle.com/datasets/paramaggarwal/fashion-product-images-small
License(s): copyright-authors
 96% 545M/565M [00:04<00:00, 140MB/s]
100% 565M/565M [00:04<00:00, 145MB/s]


In [5]:
! unzip /content/fashion-product-images-small.zip

Streaming output truncated to the last 5000 lines.
  inflating: myntradataset/images/5813.jpg  
  inflating: myntradataset/images/58131.jpg  
  inflating: myntradataset/images/58132.jpg  
  inflating: myntradataset/images/58133.jpg  
  inflating: myntradataset/images/58135.jpg  
  inflating: myntradataset/images/58136.jpg  
  inflating: myntradataset/images/58137.jpg  
  inflating: myntradataset/images/58138.jpg  
  inflating: myntradataset/images/58139.jpg  
  inflating: myntradataset/images/5814.jpg  
  inflating: myntradataset/images/58140.jpg  
  inflating: myntradataset/images/58141.jpg  
  inflating: myntradataset/images/58143.jpg  
  inflating: myntradataset/images/58144.jpg  
  inflating: myntradataset/images/58145.jpg  
  inflating: myntradataset/images/58146.jpg  
  inflating: myntradataset/images/58147.jpg  
  inflating: myntradataset/images/58148.jpg  
  inflating: myntradataset/images/58149.jpg  
  inflating: myntradataset/images/5815.jpg  
  inflating: myntradataset/image

# generate input file

In [6]:
import pandas as pd
import numpy as np
from skimage.io import imread
import os


styles_path = '/content/myntradataset/styles.csv'
images_dir = '/content/myntradataset/images'


styles_df = pd.read_csv(styles_path, nrows=5000, on_bad_lines ='skip')





In [28]:
styles_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt


In [7]:
def preprocess_data(styles_df):
    styles_df['image_path'] = styles_df['id'].apply(lambda x: os.path.join(images_dir, f"{x}.jpg"))
    return styles_df

In [30]:
styles_df = preprocess_data(styles_df)

In [8]:
from skimage.transform import resize
def load_image(image_id):
    image_path = os.path.join(images_dir, f"{image_id}.jpg")
    image = imread(image_path)
    return image

In [9]:
def image_to_flat_line(image, image_id, labels):
    pixels = image.flatten()
    label_str = ','.join(map(str, labels))
    pixel_str = ','.join(map(str, pixels))
    return f"{image_id},{label_str},{pixel_str}"

In [10]:
def create_flat_data(styles_df):
    lines = []
    for idx, row in styles_df.iterrows():
        image_id = row['id']
        labels = [row['articleType'], row['baseColour'], row['season'], row['gender']]
        image = load_image(image_id)
        if(len(image.flatten())== 14400):
           line = image_to_flat_line(image, image_id, labels)
           lines.append(line)
    return lines

In [11]:
flat_data = create_flat_data(styles_df)

In [12]:
input_file = 'input.txt'
with open(input_file, 'w') as f:
    for line in flat_data:
        f.write(line + '\n')


In [14]:
flat_data[0]

'15970,Shirts,Navy Blue,Fall,Men,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,254,255,255,254,255,255,255,255,255,255,255,255,255,254,255,255,254,255,255,254,253,255,254,253,255,249,249,255,253,253,255,254,255,255,253,254,255,253,255,255,253,255,255,254,255,247,246,251,255,255,255,254,255,255,242,242,242,248,248,248,255,254,255,254,252,253,246,242,241,255,254,253,255,254,255,255,254,255,255,254,255,255,254,255,255,254,255,255,254,255,254,254,255,254,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,255,254,255,255,254,255,255,255,255,255,255,255,255,255,254

In [40]:
len(flat_data[0].split(',')[5:])

14400

# Map reduce baby

In [1]:
from dask.distributed import Client
client = Client(n_workers=10, threads_per_worker=1, processes=True)

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.diskutils:Found stale lock file and directory '/tmp/dask-scratch-space/scheduler-jl34_kgx', purging
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:43965
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:33727'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36025'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:37927'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39331'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46021'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44935'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35605'
INFO:distributed.nanny:        Start Nanny at: 'tcp://

In [2]:
import dask.bag as db
input_file = 'input.txt'
bag = db.read_text(input_file).map(lambda x: x.split(','))

In [3]:
import dask.bag as db
from dask import delayed
def map_function_bag(data):
    print(data)
    image_id = data[0]
    labels = data[1:5]
    pixels = np.array(data[5:], dtype=np.float32)
    print("hi",len(pixels))
    return image_id, labels, pixels

mapped_bag = bag.map(map_function_bag)

In [4]:
import hashlib
def create_nft_id(image_id, closest_image_id, labels):
    label_str = ','.join(labels)
    data_str = f"{image_id},{closest_image_id},{label_str}"
    return hashlib.sha256(data_str.encode()).hexdigest()

In [5]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
def find_closest_image(image_pixels, all_pixels, current_index):
    distances = euclidean_distances([image_pixels], all_pixels)
    distances[0, current_index] = np.inf  # Set the distance to itself to infinity to avoid self-matching
    min_index = np.argmin(distances)
    min_distance = distances[0][min_index]
    return min_index, min_distance


INFO:distributed.core:Event loop was unresponsive in Nanny for 3.31s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Scheduler for 3.28s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.41s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.43s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
INFO:distributed.core:Event loop was unresponsive in Nanny for 3.43s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeout

In [9]:
from tqdm import tqdm

def reduce_function_bag(mapped_bag):
    mapped_data = mapped_bag.compute()
    all_pixels = np.stack([data[2] for data in mapped_data])
    results = []

    for current_index, data in enumerate(tqdm(mapped_data, desc="Reducing", unit="image")):
        image_id, labels, image_pixels = data
        closest_index, closest_distance = find_closest_image(image_pixels, all_pixels, current_index)
        closest_image_id = mapped_data[closest_index][0]
        nft_id = create_nft_id(image_id, closest_image_id, labels)
        results.append((image_id, closest_image_id, closest_distance, nft_id))


    return results

In [10]:
reduced_data_bag = reduce_function_bag(mapped_bag)

INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:41653', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:41653
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:44668
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:43179', name: 4, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:43179
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:44676
Reducing: 100%|██████████| 4952/4952 [55:05<00:00,  1.50image/s]


In [12]:
reduced_data_bag[0]

('15970',
 '14054',
 4476.453,
 '5fac6d407c62b5904395504e52df48fec477d6832a8a987986c4f248e916797d')

In [13]:
output_file = 'output.txt'
with open(output_file, 'w') as f:
    for result in reduced_data_bag:
        image_id, closest_image_id, closest_distance, nft_id = result
        f.write(f"{image_id},{closest_image_id},{closest_distance},{nft_id}\n")